In [1]:
#--mesure time
import time 
#soundnet in pytorch
import extract_features as ex
import pandas as pd 
import librosa
from tqdm import tqdm
import numpy as np

In [2]:
from sklearn.cluster import SpectralClustering
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

In [3]:
import numpy as np
def extract_vector(features,layer):
    # C : channel_output and H : Height
    # Vector's shape (H,C)
    C = features[layer][0].shape[0]    
    print("Cout : ",C)
    return (features[layer].reshape(C,-1).T)


In [4]:

def extract_vectors(filepath):
    audio,sr = ex.load_audio(filepath)
    features = ex.extract_pytorch_feature(audio,'./sound8.pth')
   
    print([x.shape for x in features])
    
    ##extract vector
    vectors = []
    for idlayer in range(len(features)):
        vectors.append(ex.extract_vector(features,idlayer)) #features vector 
    return vectors, len(audio)/sr

In [5]:
#FUNCTION to align all features vectors

def quantize_test(vector,idlayer, res, size):
    to_layer = {}
    for i in range(9):
        to_layer[i] = "conv{}".format(i+1)

    lim = int(size/res)
    m,b = ex.get_values()[to_layer[idlayer]] #get slope (m) and interception (b) for a given layer
    vectimes = (np.array(range(0,len(vector))))/m
    maxtime = (len(vector))*(1/m)
    alltimes = np.arange(0,maxtime,res)
    qvec = []
    for i,curtime in enumerate(alltimes[:-1]):
        ind = np.argwhere((vectimes>=curtime) & (vectimes<alltimes[i+1]))
        qvec.append(np.median(vector[ind],axis = 0).reshape(-1))
    
    data = (np.stack(qvec))
    #meme taille
    dif = lim - len(data)
    if dif < 0:
        data = data[:lim]
    elif dif > 0:
        x = np.tile(data[-1],(dif,1))
        data = np.concatenate([data,x])        
    return data
    


In [7]:
filepath = 'sherlockaudio.wav'
vector,size = extract_vectors(filepath)

time:  0.15762543678283691
Tensor shape: torch.Size([1, 1, 62609408, 1])
[(1, 16, 3913088, 1), (1, 32, 244568, 1), (1, 64, 122285, 1), (1, 128, 61143, 1), (1, 256, 7643, 1), (1, 512, 3822, 1), (1, 1024, 1912, 1), (1, 1000, 953, 1), (1, 401, 953, 1)]


In [ ]:
#conv1 to con7è 
allfeatures = []
for i in range(7):
    feature = quantize_test(vector[i],i,1.5,size)
    print(feature.shape)
    np.savez_compressed('./soundnet_features/sherlock_pytorch/conv{}.npz'.format(i+1),fv = feature)